# Baseline model - no debiasing

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten,Dropout, LeakyReLU, Rescaling
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
df = pd.read_csv('../input/dataframes/train_df_kaggle.csv')
df

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
train_df = df[0:20000]
train_df

In [ ]:
val_df = df[20000:30000]
val_df

In [ ]:
# test_df = pd.read_csv('/content/drive/MyDrive/BE Project/D/test_df.csv')

In [ ]:
IMAGE_SIZE=[224,224]

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255,
)

train_set = train_generator.flow_from_dataframe(
    train_df, 
    directory = "../input/croppedframes/cropped frames/cropped frames/", 
    x_col='Video',
#     y_col='O',
    y_col=['O','C','E','A','N'],
    target_size=IMAGE_SIZE,
#     subset = 'training',
    color_mode ='rgb',
    class_mode='raw',
    shuffle = False,
    # batch_size=1024
)

In [ ]:
val_generator = ImageDataGenerator(
    rescale=1./255,
)

validation_set = val_generator.flow_from_dataframe(
    val_df, 
    directory = "../input/croppedframes/cropped frames/cropped frames/", 
    x_col='Video',
#     y_col='O',
    y_col=['O','C','E','A','N'],
    target_size=IMAGE_SIZE,
#     subset = 'validation',
    color_mode ='rgb',
    class_mode='raw',
    shuffle = True,
    # batch_size=1024
)

In [ ]:
# test_generator = ImageDataGenerator(
#     rescale=1./255,
# )

# test_set = test_generator.flow_from_dataframe(
#     test_df, 
#     "", 
#     x_col='Video',
#     y_col= None,
#     target_size=IMAGE_SIZE,
#     color_mode ='rgb',
#     class_mode=None,
#     shuffle = False,
# )

In [ ]:
# def generator_wrapper(generator):
#     for batch_x,batch_y in generator:
#         yield (batch_x,[batch_y[:,i] for i in range(5)])

In [ ]:
early = EarlyStopping(patience=20)

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

    
x = Flatten()(vgg.output)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)

prediction = Dense(5, activation='sigmoid')(x)
# o = Dense(1, activation='sigmoid', name='o')(x)
# c = Dense(1, activation='sigmoid', name='c')(x)
# e = Dense(1, activation='sigmoid', name='e')(x)
# a = Dense(1, activation='sigmoid', name='a')(x)
# n = Dense(1, activation='sigmoid', name='n')(x)

# create a model object
# model = Model(inputs=vgg.input, outputs=[o,c,e,a,n])
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

optimizer = Adam(learning_rate=0.0001)

model.compile(loss='mae', optimizer=optimizer, metrics='mae')

# tell the model what cost and optimization method to use
# model.compile(
#   loss={'o':'mae',
#         'c':'mae',
#         'e':'mae',
#         'a':'mae',
#         'n':'mae',
#        },
#   optimizer=optimizer,
#   metrics={'o':'mae',
#         'c':'mae',
#         'e':'mae',
#         'a':'mae',
#         'n':'mae',
#   },
# )

In [ ]:
# STEP_SIZE_TRAIN=train_set.n//train_set.batch_size
# STEP_SIZE_VALID=validation_set.n//validation_set.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

# model.fit_generator(generator=generator_wrapper(train_set), 
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=generator_wrapper(validation_set),
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=50,
#                     shuffle = True,
#                     callbacks = [early])

r = model.fit_generator(
  train_set,
  steps_per_epoch=len(train_set),
  epochs=50,
  validation_data=validation_set,
  validation_steps=len(validation_set),
  shuffle = True,
  callbacks = [early],
)

In [ ]:
model.save('model_biased_mae.h5')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink 
FileLink(r'model_biased_mae.h5')

# Debiasing

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Flatten,Dropout, LeakyReLU, Rescaling, BatchNormalization

In [3]:
IMAGE_SIZE = [224,224]

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False


classifier = keras.Sequential(
    [
        vgg,
        Flatten(),
        Dense(4096, activation='relu', name='dense'),
#         Dense(4096, activation=keras.layers.ELU(), name='dense'),
#         BatchNormalization(axis = 1),
        Dropout(0.5),
        Dense(5, activation='sigmoid', name="output"),
    ],
    name="classifier",
)


adversary = keras.Sequential(
    [
        keras.Input(shape=(5,)),
        Dense(200, activation='relu', name="dense"),
#         BatchNormalization(axis = 1),
#         Dense(200, activation=keras.layers.ELU(), name="dense"),
        Dense(2, activation='sigmoid', name="output"),
    ],
    name="adversary",
)

In [5]:
class AdversarialDebiasing(keras.Model):
    def __init__(self, classifier, adversary, alpha, c_loss, a_loss, debias=True):
        super(AdversarialDebiasing, self).__init__()
        self.classifier = classifier
        self.adversary = adversary
        self.c_loss = c_loss #metric for classifier
        self.a_loss = a_loss #metric for adversaary
        self.protect_loss_weight = alpha
        self.debias = debias
        
    @property
    def metrics(self):
        return [self.c_loss, self.a_loss]


    def compile(self, optimizer,c_loss_fn, a_loss_fn):
        super(AdversarialDebiasing, self).compile()
        self.c_optimizer = optimizer[0]
        self.a_optimizer = optimizer[1]
        self.c_loss_fn = c_loss_fn
        self.a_loss_fn = a_loss_fn

        
    def call(self, data):
        x = data
        y = self.classifier(x)
        z = self.adversary(y)
        return [y,z]
        
        
    def train_step(self, data):
        
        x, y = data
        
        e_g = y[1]
        y = y[0]

        with tf.GradientTape() as tape:
            c_predictions = self.classifier(x)
            c_loss = self.c_loss_fn(y, c_predictions)

            
        c_grads = tape.gradient(c_loss, self.classifier.trainable_weights)

        
        with tf.GradientTape() as tape:
            c_predictions = self.classifier(x)
            a_predictions = self.adversary(c_predictions)
            a_loss = self.a_loss_fn(e_g, a_predictions)
            
        
        a_grads = tape.gradient(a_loss, self.classifier.trainable_weights) #projection
        
        
        with tf.GradientTape() as tape:
            c_predictions = self.classifier(x)
            a_predictions = self.adversary(c_predictions)
            a_loss = self.a_loss_fn(e_g, a_predictions)
            
        a_grads_own = tape.gradient(a_loss, self.adversary.trainable_weights)

        if self.debias:
            protect_grad = {v.name: g for (g, v) in zip(a_grads, self.classifier.trainable_weights)}
            pred_grad = [] #classifier update function
        
            for (g, v) in zip(c_grads, self.classifier.trainable_weights):
                unit_protect = protect_grad[v.name] / (tf.norm(protect_grad[v.name]) + np.finfo(np.float32).tiny)
                g -= tf.reduce_sum(g * unit_protect) * unit_protect # g- projection
                g -= self.protect_loss_weight * protect_grad[v.name] # g - projection - alpha*adv grad
                pred_grad.append((g, v))
                 
            self.c_optimizer.apply_gradients(pred_grad)
        
        else:
            self.c_optimizer.apply_gradients(zip(c_grads, self.classifier.trainable_weights))
            
        
        self.a_optimizer.apply_gradients(zip(a_grads_own, self.adversary.trainable_weights))
        
        self.c_loss.update_state(y,c_predictions)
        self.a_loss.update_state(e_g, a_predictions)
        
        return {m.name: m.result() for m in self.metrics}
    
    
    
    def test_step(self, data):
        
        x, y = data
        
        e_g = y[1]
        y = y[0]

        c_predictions = self.classifier(x)
        c_loss = self.c_loss_fn(y, c_predictions)
        a_predictions = self.adversary(c_predictions)
        a_loss = self.a_loss_fn(e_g, a_predictions)
            
        
        self.c_loss.update_state(y,c_predictions)
        self.a_loss.update_state(e_g, a_predictions)
        
        return {m.name: m.result() for m in self.metrics}

In [6]:
df = pd.read_csv('../input/dataframes/train_df_kaggle.csv', nrows=30000)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.sort_values(by=['Video'], inplace=True)

In [7]:
train_df = df[0:20000]
# val_df = pd.read_csv('../input/dataframes/train_df_kaggle.csv', nrows=10000, skiprows=20000)
val_df = df[20000:30000]

In [8]:
ethnicity_gender = pd.read_csv('../input/croppedframes/eth_gender_annotations_dev.csv', delimiter = ';')
ethnicity_gender.sort_values(by=['VideoName'])
ethnicity_gender

In [9]:
def get_video_name(data):
    data['VideoName'] = data['Video'].str.slice(0,15) + '.mp4'

In [10]:
def merge_protected_attributes(ethnicity_gender, data):
    data = pd.merge(data, ethnicity_gender, on='VideoName', how='inner')
    data = data[['Video','O','C','E','A', 'N', 'Ethnicity', 'Gender']]
    return data

In [11]:
get_video_name(train_df)
train_df = merge_protected_attributes(ethnicity_gender, train_df)
train_df

In [12]:
get_video_name(val_df)
val_df = merge_protected_attributes(ethnicity_gender, val_df)
val_df

In [13]:
train_df.loc[(train_df['Ethnicity']==1) | (train_df['Ethnicity']==3), 'Ethnicity'] = 0
train_df.loc[train_df['Ethnicity']==2, 'Ethnicity'] = 1

train_df.loc[(train_df['Gender']==1), 'Gender'] = 1
train_df.loc[train_df['Gender']==2, 'Gender'] = 0

train_df

In [14]:
val_df.loc[(val_df['Ethnicity']==1) | (val_df['Ethnicity']==3), 'Ethnicity'] = 0
val_df.loc[val_df['Ethnicity']==2, 'Ethnicity'] = 1

val_df.loc[(val_df['Gender']==1), 'Gender'] = 1
val_df.loc[val_df['Gender']==2, 'Gender'] = 0

train_df

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale=1./255,
)

val_generator = ImageDataGenerator(
    rescale=1./255,
)

In [16]:
def generate_data_generator(generator, df):
    genX1 = generator.flow_from_dataframe(
    df, 
    directory = "../input/croppedframes/cropped frames/cropped frames/", 
    x_col='Video',
    y_col=['O','C','E','A','N'],
    target_size=IMAGE_SIZE,
    color_mode ='rgb',
    class_mode='raw',
    shuffle = False,
    )
    
    genX2 =generator.flow_from_dataframe(
        df, 
        directory = "../input/croppedframes/cropped frames/cropped frames/", 
        x_col='Video',
        y_col=['Ethnicity','Gender'],
        target_size=IMAGE_SIZE,
        color_mode ='rgb',
        class_mode='raw',
        shuffle = False,
    )
    
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield X1i[0], [X1i[1], X2i[1]]

In [17]:
adv_cls = AdversarialDebiasing(classifier, adversary, 1,
                               keras.metrics.MeanAbsoluteError(name="c_loss"), 
#                                keras.metrics.MeanAbsoluteError(name="a_loss") 
#                                keras.metrics.BinaryCrossentropy(name="c_loss"),
                               keras.metrics.BinaryCrossentropy(name="a_loss"),
                               debias=False,
                              )


adv_cls.compile(
#     optimizer=[keras.optimizers.Adam(learning_rate=0.0001),keras.optimizers.Adam(learning_rate=0.0001),],
    optimizer=[keras.optimizers.SGD(nesterov=True),keras.optimizers.SGD(nesterov=True),],
#     c_loss_fn=keras.losses.BinaryCrossentropy(),
    c_loss_fn = keras.losses.MeanAbsoluteError(),
    a_loss_fn = keras.losses.BinaryCrossentropy(),
)

In [ ]:
history = adv_cls.fit(generate_data_generator(train_generator, train_df), 
                      epochs=50, 
                      steps_per_epoch=len(train_df) / 32,
                      validation_data = generate_data_generator(val_generator, val_df),
                      validation_steps = len(val_df) / 32,
                     )

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['c_loss'])
plt.plot(history.history['a_loss'])

In [ ]:
adv_cls.save_weights('adv_model_debias_false.h5')

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink 
FileLink(r'adv_model_debias_false.h5')

# Prediction and metrics

In [24]:
!pip install aif360

In [25]:
!pip install fairlearn

In [26]:
!pip install --upgrade scipy

In [27]:
from aif360.sklearn import metrics

In [34]:
def fairness_metrics(data, trait, pos):
    trait_data = data[[trait]].copy()
#     print(trait_data.columns)
#     trait_data.set_index([trait_data.index,'Ethnicity', 'Gender'], inplace=True)
    trait_data.loc[trait_data[trait] < 0.5, trait] = 0
    trait_data.loc[trait_data[trait] >= 0.5, trait] = 1
    
    
    trait_data_pred = data[[trait+'_pred']].copy()
#     print(trait_data_pred.columns)
#     trait_data_pred.set_index([trait_data.index,'Ethnicity', 'Gender'], inplace=True)
    trait_data_pred.loc[trait_data_pred[trait+'_pred'] < 0.5, trait+'_pred'] = 0
    trait_data_pred.loc[trait_data_pred[trait+'_pred'] >= 0.5, trait+'_pred'] = 1
  
  
    print("Ethnicity")
    print(metrics.disparate_impact_ratio(trait_data_pred, 
                                      prot_attr='Ethnicity',
                                    priv_group=1,
                                      pos_label=pos))
  
    #     print(metrics.statistical_parity_difference(trait_data_pred, 
#                                               prot_attr='Ethnicity',
#                                         priv_group=1,
#                                               pos_label=pos))
  
#     print(metrics.equal_opportunity_difference(trait_data,
#                                              trait_data_pred[trait+'_pred'], 
#                                              prot_attr='Ethnicity',
#                                             priv_group=1,
#                                              pos_label=pos))
  
#     print(metrics.average_odds_difference(trait_data,
#                                         trait_data_pred, 
#                                         prot_attr='Ethnicity',
#                                         priv_group=1,
#                                         pos_label=pos))
    
    
    print("Gender")
    print(metrics.disparate_impact_ratio(trait_data_pred, 
                                      prot_attr='Gender', 
                                      priv_group=1, 
                                      pos_label=pos))
  
    #     print(metrics.statistical_parity_difference(trait_data_pred, 
#                                               prot_attr='Gender', 
#                                               priv_group=1, 
#                                               pos_label=pos))
  
#     print(metrics.equal_opportunity_difference(trait_data,
#                                              trait_data_pred[trait+'_pred'], 
#                                              prot_attr='Gender', 
#                                              priv_group=1, 
#                                              pos_label=pos))
  
#     print(metrics.average_odds_difference(trait_data,
#                                         trait_data_pred, 
#                                         prot_attr='Gender',
#                                         priv_group=1, 
#                                         pos_label=pos))

In [ ]:
def statistical_parity(df, threshold, trait):
    protected_attr = 'Ethnicity'
    trait_df = df[[trait+'_pred', protected_attr]]
    
    total_count = trait_df.shape[0]
    above_thresh_count = trait_df[trait_df[trait+'_pred']>=threshold].shape[0]
    
    total_unpriv_count = trait_df[trait_df[protected_attr]==0].shape[0]
    above_thresh_unpriv_count = trait_df[(trait_df[trait+'_pred']>=threshold) & (trait_df[protected_attr]==0)].shape[0]
    
    P_total = above_thresh_count/total_count
    P_protected = above_thresh_unpriv_count/ total_unpriv_count
    
    print('Ethnicity-0' ,P_total-P_protected)
    
    total_count = trait_df.shape[0]
    above_thresh_count = trait_df[trait_df[trait+'_pred']>=threshold].shape[0]
    
    total_unpriv_count = trait_df[trait_df[protected_attr]==1].shape[0]
    above_thresh_unpriv_count = trait_df[(trait_df[trait+'_pred']>=threshold) & (trait_df[protected_attr]==1)].shape[0]
    
    P_total = above_thresh_count/total_count
    P_protected = above_thresh_unpriv_count/ total_unpriv_count
    print('Ethnicity-1' ,P_total-P_protected)
    
    
    
    protected_attr = 'Gender'
    trait_df = df[[trait+'_pred', protected_attr]]
    
    total_count = trait_df.shape[0]
    above_thresh_count = trait_df[trait_df[trait+'_pred']>=threshold].shape[0]
    
    total_unpriv_count = trait_df[trait_df[protected_attr]==0].shape[0]
    above_thresh_unpriv_count = trait_df[(trait_df[trait+'_pred']>=threshold) & (trait_df[protected_attr]==0)].shape[0]
    
    P_total = above_thresh_count/total_count
    P_protected = above_thresh_unpriv_count/ total_unpriv_count
    print('Gender-0', P_total-P_protected)
    
    total_count = trait_df.shape[0]
    above_thresh_count = trait_df[trait_df[trait+'_pred']>=threshold].shape[0]
    
    total_unpriv_count = trait_df[trait_df[protected_attr]==1].shape[0]
    above_thresh_unpriv_count = trait_df[(trait_df[trait+'_pred']>=threshold) & (trait_df[protected_attr]==1)].shape[0]
    
    P_total = above_thresh_count/total_count
    P_protected = above_thresh_unpriv_count/ total_unpriv_count
    print('Gender-1', P_total-P_protected)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
def bounded_group_loss(df, trait):
    
    trait_df = df[[trait, trait+'_pred', 'Ethnicity']]
    #ethnicity
    eth_0 = trait_df[trait_df['Ethnicity']==0]
    mae_0 = mae(eth_0[trait].tolist(), eth_0[trait+'_pred'].tolist())
    print('MAE for Ethnicity = 0', mae_0)
    
    eth_1 = trait_df[trait_df['Ethnicity']==1]
    mae_1 = mae(eth_1[trait].tolist(), eth_1[trait+'_pred'].tolist())
    print('MAE for Ethnicity = 1', mae_1)
    
    #gender
    trait_df = df[[trait, trait+'_pred', 'Gender']]
    gender_0 = trait_df[trait_df['Gender']==0]
    mae_0 = mae(gender_0[trait].tolist(), gender_0[trait+'_pred'].tolist())
    print('MAE for Gender = 0', mae_0)
    
    gender_1 = trait_df[trait_df['Gender']==1]
    mae_1 = mae(gender_1[trait].tolist(), gender_1[trait+'_pred'].tolist())
    print('MAE for Gender = 1', mae_1)

In [1]:
adv_cls = AdversarialDebiasing(classifier, adversary, 1,
                               keras.metrics.MeanAbsoluteError(name="c_loss"), 
#                                keras.metrics.MeanAbsoluteError(name="a_loss") 
#                                keras.metrics.BinaryCrossentropy(name="c_loss"),
                               keras.metrics.BinaryCrossentropy(name="a_loss"),
                               debias=True
                              )

adv_cls.compile(
    optimizer=[keras.optimizers.Adam(learning_rate=0.0001),keras.optimizers.Adam(learning_rate=0.0001),],
#     c_loss_fn=keras.losses.BinaryCrossentropy(),
    c_loss_fn = keras.losses.MeanAbsoluteError(),
    a_loss_fn = keras.losses.BinaryCrossentropy(),
)

In [18]:
adv_cls.built = True
adv_cls.load_weights('../input/models/adv_model_debias_false.h5')

In [19]:
op_adv = adv_cls.predict(generate_data_generator(train_generator, train_df), steps =len(train_df) / 32)

In [ ]:
model = keras.models.load_model('../input/models/model_biased_mae.h5')

In [ ]:
op = model.predict(train_set)

In [20]:
metric_check = train_df[['O', 'C', 'E', 'A', 'N', 'Ethnicity', 'Gender']]
metric_check

In [21]:
metric_check.loc[:, ['O_pred', 'C_pred', 'E_pred', 'A_pred', 'N_pred']] = op_adv[0]
# metric_check.loc[:, ['O_pred', 'C_pred', 'E_pred', 'A_pred', 'N_pred']] = op

In [22]:
metric_check.set_index([metric_check.index, 'Ethnicity', 'Gender'], inplace=True)
metric_check

In [35]:
fairness_metrics(metric_check[['O', 'O_pred']], 'O', 0)
fairness_metrics(metric_check[['O', 'O_pred']], 'O', 1)

In [36]:
fairness_metrics(metric_check[['C', 'C_pred']], 'C', 0)
fairness_metrics(metric_check[['C', 'C_pred']], 'C', 1)

In [37]:
fairness_metrics(metric_check[['E', 'E_pred']], 'E', 0)
fairness_metrics(metric_check[['E', 'E_pred']], 'E', 1)

In [38]:
fairness_metrics(metric_check[['A', 'A_pred']], 'A', 0)
fairness_metrics(metric_check[['A', 'A_pred']], 'A', 1)

In [39]:
fairness_metrics(metric_check[['N', 'N_pred']], 'N', 0)
fairness_metrics(metric_check[['N', 'N_pred']], 'N', 1)

In [ ]:
statistical_parity(metric_check, 0.5, 'O')

In [ ]:
statistical_parity(metric_check, 0.5, 'C')

In [ ]:
statistical_parity(metric_check, 0.5, 'E')

In [ ]:
statistical_parity(metric_check, 0.5, 'A')

In [ ]:
statistical_parity(metric_check, 0.5, 'N')

In [ ]:
bounded_group_loss(metric_check, 'O')

In [ ]:
bounded_group_loss(metric_check, 'C')

In [ ]:
bounded_group_loss(metric_check, 'E')

In [ ]:
bounded_group_loss(metric_check, 'A')

In [ ]:
bounded_group_loss(metric_check, 'N')

In [ ]:
# def fairness(df, trait):
#     trait_df = df[[trait, trait+'_pred', 'Ethnicity']]
#     #ethnicity
#     eth_0 = trait_df[trait_df['Ethnicity']==0]
#     eth_1 = trait_df[trait_df['Ethnicity']==1]
    
#     sum_0 = eth_0[trait+'_pred'].sum()
#     sum_1 = eth_1[trait+'_pred'].sum()
    
#     print(sum_0/eth_0.shape[0] - sum_1/eth_1.shape[0])
    
#     #gender
#     trait_df = df[[trait, trait+'_pred', 'Gender']]
#     #ethnicity
#     gender_0 = trait_df[trait_df['Gender']==0]
#     gender_1 = trait_df[trait_df['Gender']==1]
    
#     sum_0 = gender_0[trait+'_pred'].sum()
#     sum_1 = gender_1[trait+'_pred'].sum()
    
#     print(sum_0/gender_0.shape[0] - sum_1/gender_1.shape[0])